This notebook use native function of SK to call Language Service, if not found we generate using RAG

In [1]:
#r "nuget: Azure.Core, 1.39.0"
#r "nuget: Azure.AI.Language.QuestionAnswering, 1.1.0"
#r "nuget: Microsoft.SemanticKernel, 1.17.1"

Installed Packages Azure.AI.Language.QuestionAnswering, 1.1.0 Azure.Core, 1.39.0 Microsoft.SemanticKernel, 1.17.1

Load all settings from env file

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

void LoadEnv()
{
    string filename = ".env";

    if (!File.Exists(filename))
        return;

    foreach (var line in File.ReadAllLines(filename))
    {
        var parts = line.Split(
            '=',
            StringSplitOptions.RemoveEmptyEntries);

        if (parts.Length != 2)
            continue;

        Environment.SetEnvironmentVariable(parts[0], parts[1]);
    }
}

LoadEnv();

Create client for Language Service

In [23]:
using Azure;
using Azure.AI.Language.QuestionAnswering;

Uri endpoint = new Uri(Environment.GetEnvironmentVariable("LANGUAGE_SERVICE_ENDPOINT"));
AzureKeyCredential credential = new AzureKeyCredential(Environment.GetEnvironmentVariable("LANGUAGE_SERVICE_KEY"));
string projectName = Environment.GetEnvironmentVariable("PROJECT_NAME");
string deploymentName = "production";

QuestionAnsweringClient client = new (endpoint, credential);
QuestionAnsweringProject project = new (projectName, deploymentName);

Now let test if this work

In [ ]:
string question = "What is the capital of Canada ?";

Response<AnswersResult> response = await client.GetAnswersAsync(question,project);

foreach (KnowledgeBaseAnswer answer in response.Value.Answers)
{
    Console.WriteLine($"Q:{question}");
    Console.WriteLine($"A:{answer.Answer}");
}

Now let's create our SK native function

In [3]:
using System.Text.Json.Serialization;
using System.ComponentModel;
using Azure;
using Azure.AI.Language.QuestionAnswering;

public class KBAnswer
{
    [JsonPropertyName("answers")]
    public List<string> Answers { get; set; }

    public KBAnswer()
    {
        Answers = new List<string>();
    }
}

public class KBPlugin
{
   QuestionAnsweringClient _client;
   QuestionAnsweringProject _project;

   public KBPlugin()
   {
        // This is only for test purposes in production you use dependency injection
        Uri endpoint = new Uri(Environment.GetEnvironmentVariable("LANGUAGE_SERVICE_ENDPOINT"));
        AzureKeyCredential credential = new AzureKeyCredential(Environment.GetEnvironmentVariable("LANGUAGE_SERVICE_KEY"));
        string projectName = Environment.GetEnvironmentVariable("PROJECT_NAME");
        string deploymentName = "production";

        _client = new (endpoint, credential);
        _project = new (projectName, deploymentName);
   }

   [KernelFunction("get_answer_kb")]
   [Description("Get an answer from the KB")]
   [return: Description("All answers found in the KB")]
   public async Task<KBAnswer> GetAnswerKB(string question) 
   {
        Response<AnswersResult> response = await _client.GetAnswersAsync(question,_project);

        var kb = new KBAnswer();

        foreach (KnowledgeBaseAnswer answer in response.Value.Answers)
        {
            kb.Answers.Add(answer.Answer);            
        }

        return kb;
   }
}

Now let's build the kernel

In [4]:
var deployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_CHAT_DEPLOYMENT_MODEL");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
var apiEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");

var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(deployment,apiEndpoint,apiKey)                   
                   .Build();

var chat = kernel.GetRequiredService<IChatCompletionService>();

// Add the plugin to the kernel

kernel.Plugins.AddFromType<KBPlugin>("KB");

// Enable planning
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

Now let's test if this work

In [5]:
var history = new ChatHistory();

history.AddUserMessage("What is the capital of Canada?");

var response = await chat.GetChatMessageContentAsync(history,
                                                     executionSettings: openAIPromptExecutionSettings,
                                                     kernel: kernel);

Console.WriteLine(response);

The capital of Canada is Ottawa.


Now let create another SK native function that will return answer from dishes in Canada (this is not present in the KB)

In [6]:
using System.Text.Json.Serialization;
using System.ComponentModel;
using Azure;
using Azure.AI.Language.QuestionAnswering;
using System.Text.RegularExpressions;

public class Dish
{
    [JsonPropertyName("name")]
    public string Name { get; set; }

    [JsonPropertyName("description")]
    public string Description { get; set; }

    [JsonPropertyName("province")]
    public string Province { get; set; }
}

public class DishPlugin
{
   List<Dish> _dishes;

   public DishPlugin()
   {
        _dishes = new List<Dish>()
        {
            new Dish { Name = "Poutine", Description = "A popular Canadian dish made of french fries, cheese curds, and gravy.", Province = "Quebec" },
            new Dish { Name = "Butter Tart", Description = "A sweet pastry filled with a gooey buttery filling, often made with maple syrup.", Province = "Ontario" },
            new Dish { Name = "Nanaimo Bar", Description = "A no-bake dessert bar made of a chocolate coconut crumb base, custard filling, and chocolate ganache topping.", Province = "British Columbia" },
            new Dish { Name = "BeaverTails", Description = "A fried dough pastry stretched to resemble a beaver's tail, often topped with various sweet toppings.", Province = "Ontario" },
            new Dish { Name = "Tourtière", Description = "A traditional meat pie originating from Quebec, typically filled with ground pork, beef, or veal.", Province = "Quebec" },
            new Dish { Name = "Too Many Bones", Description = "Bog are excellent in too many bones, this is popular in quebec made from Bog", Province = "Quebec" }
        };
   }

   [KernelFunction("get_popular_dishes")]
   [Description("Get popular dished from Canada")]
   [return: Description("Popular dishes from Canada based by province")]
   public async Task<List<Dish>> GetDishes(string question) 
   {
        string province = ExtractProvince(question);
        var filteredDishes = _dishes.Where(d => d.Province.Equals(province, StringComparison.OrdinalIgnoreCase)).ToList();

        return await Task.FromResult(filteredDishes);
   }

    private string ExtractProvince(string question)
    {
        var match = Regex.Match(question, @"\bin\s(\w+)\b", RegexOptions.IgnoreCase);
        return match.Success ? match.Groups[1].Value : string.Empty;
    }
}

Add the new plugin to the kernel

In [7]:
kernel.Plugins.AddFromType<DishPlugin>("Dish");

Ask popular dish from Quebec

In [8]:
var history = new ChatHistory();

history.AddUserMessage("What is popular dishes in Quebec?");

var response = await chat.GetChatMessageContentAsync(history,
                                                     executionSettings: openAIPromptExecutionSettings,
                                                     kernel: kernel);

Console.WriteLine(response);

Some popular dishes in Quebec are:

1. Poutine: A popular Canadian dish made of french fries, cheese curds, and gravy.
2. Tourtière: A traditional meat pie originating from Quebec, typically filled with ground pork, beef, or veal.
3. Too Many Bones: This is a unique dish popular in Quebec, made from Bog.

These are just a few examples, and there are many more delicious dishes to explore in Quebec.


Now let's ask from Mexico, since we didn't limit the LLM to our context only it should find an answer

In [9]:
var history = new ChatHistory();

history.AddUserMessage("What is popular dishes in Mexico?");

var response = await chat.GetChatMessageContentAsync(history,
                                                     executionSettings: openAIPromptExecutionSettings,
                                                     kernel: kernel);

Console.WriteLine(response);

I'm sorry, but I couldn't find any specific information about popular dishes in Mexico. However, some well-known Mexican dishes include tacos, enchiladas, tamales, guacamole, and chiles rellenos. These are just a few examples, and there are many more delicious dishes in Mexican cuisine.


Now let tell them if not found in native function do nothing

In [10]:
var history = new ChatHistory();

history.AddAssistantMessage("If you cannot find the answer from your native function reply I don't have the information, sorry");
history.AddUserMessage("What is popular dishes in Mexico?");

var response = await chat.GetChatMessageContentAsync(history,
                                                     executionSettings: openAIPromptExecutionSettings,
                                                     kernel: kernel);

Console.WriteLine(response);

I'm sorry, but I don't have the information on popular dishes in Mexico.
